In [1]:
import requests
import json
import time
from PIL import Image
import pandas as pd
import googlemaps

In [2]:
# Collecting location information:
url = 'http://ipinfo.io/json'

### Reference:
- https://www.youtube.com/watch?v=uowJ7LO8L90&t=74s

In [3]:
loc_json = requests.get(url)

In [5]:
my_loc = json.loads(loc_json.content)

In [6]:
curr_location = my_loc['loc']

In [7]:
curr_location

'38.8048,-77.0469'

In [22]:
# a class to use google api-key and return the places:
class GooglePlaces(object):
    def __init__(self, apikey):
        super(GooglePlaces, self).__init__()
        self.apikey = apikey
    
    def search_places_by_coordinate(self, location, radius, types):
        endpoint_rul = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
        places = []
        params = {
            'location' : location,
            'radius' : radius,
            'type' : types,
            'key' : self.apikey}
        
        res = requests.get(endpoint_rul, params = params)
        print(res)
        
        # loading to the python dictionary using json.loads function
        results = json.loads(res.content)
        
        places.extend(results['results'])
        time.sleep(2)
        while 'next_page_token' in results:
            params['pagetoken'] = results['next_page_token'],
            res = requests.get(endpoint_rul, params=params)
            results = json.loads(res.content)
            places.extend(results['results'])
            time.sleep(2)
        return places
    
    def get_place_details(self, place_id, fields):
        endpoint_url = "https://maps.googleapis.com/maps/api/place/details/json?"
        params = {
            'placeid': place_id,
            'fields': ",".join(fields),
            'key': self.apikey
        }
        res = requests.get(endpoint_url, params = params)
        place_details =  json.loads(res.content)
        return place_details
    

### Resources
- https://python.gotrained.com/google-places-api-extracting-location-data-reviews/

In [23]:
# instantiating a googleplaces class:
api = GooglePlaces('*****')

In [24]:
places = api.search_places_by_coordinate(curr_location,  "3000", "restaurant")

<Response [200]>


In [25]:
fields = ['name', 'user_ratings_total', 'formatted_address','geometry', 'international_phone_number', 'website', 'rating', 'review', 'photo']

In [26]:
api_key = '*****'

In [27]:
gmaps = googlemaps.Client(key = api_key)

In [ ]:
# Extracting images from google places:
# It will extract, 10 images from each of the nearest 60 places 
# Run when needed

In [ ]:
# for place in places:
#     details = api.get_place_details(place['place_id'], fields)
    
#     for i in range(30):
#         try:
#             photo_id = details['result']['photos'][i]['photo_reference']

#             # define dimentions
#             photo_width = 700
#             photo_height = 700

#             name = place['name']
#             raw_image_data = gmaps.places_photo(photo_reference= photo_id, max_height=photo_height,
#                                                max_width = photo_width)
#             f = open(f'./extracted_pics/{name}+{i}myimage.jpg', 'wb')
#             for each in raw_image_data:
#                 if each:
#                     f.write(each)
#             f.close()
#         except:
#             continue

In [29]:
# Creating a dataframe of the collected places:

places_df = pd.DataFrame({'name':[],'address':[],'geometry':[],'phone_number':[],
                       'website':[], 'total_user_ratings':[], 'reviews':[]})


for place in places:
    details = api.get_place_details(place['place_id'], fields)
    
    try:
        website = details['result']['website']
    except KeyError:
        website = ""
        
    try:
        name = details['result']['name']
    except KeyError:
        name = ""
    try:
        geometry = details['result']['geometry']['location']
    except:
        geometry = ""
    
    try:
        user_ratings_total = details['result']['user_ratings_total']
    except KeyError:
        user_ratings_total = ""
        
    try:
        address = details['result']['formatted_address']
    except KeyError:
        address = ""
        
    try:
        phone_number = details['result']['international_phone_number']
    except KeyError:
        phone_number = ""
        
    try:
        reviews = details['result']['reviews']
    except KeyError:
        reviews = []
        
    places_df.loc[len(places_df)] = [name, address, geometry, phone_number, website, user_ratings_total, reviews]

In [30]:
places_df.to_csv('./data/places_df.csv', index=False)

In [31]:
places_df.head()

,name,address,geometry,phone_number,website,total_user_ratings,reviews
0,Silver Diner,"6592 Springfield Mall, Springfield, VA 22150, USA","{'lat': 38.7775425, 'lng': -77.17359239999999}",+1 703-924-1701,http://www.silverdiner.com/,2484,"[{'author_name': 'Hilda Kroll', 'author_url': ..."
1,Thai Cafe,"6701 Loisdale Rd E, Springfield, VA 22150, USA","{'lat': 38.7728444, 'lng': -77.17861940000002}",+1 703-922-4942,http://www.thaicafespringfield.com/,333,"[{'author_name': 'Anne M.', 'author_url': 'htt..."
2,PaperMoon Springfield,"6315 Amherst Ave, Springfield, VA 22150, USA","{'lat': 38.78111879999999, 'lng': -77.185665}",+1 703-866-4160,http://papermoonvip.com/,229,"[{'author_name': 'Charles Lundgate', 'author_u..."
3,Chick-fil-A,"6681A Backlick Rd, Springfield, VA 22150, USA","{'lat': 38.773536, 'lng': -77.1830657}",+1 703-644-0155,https://www.chick-fil-a.com/springfieldinline,2344,"[{'author_name': 'Winter Bray', 'author_url': ..."
4,Bertucci's Italian Restaurant,"6525 Frontier Dr, Springfield, VA 22150, USA","{'lat': 38.7750714, 'lng': -77.17107779999999}",+1 703-313-6700,https://locations.bertuccis.com/us/va/springfi...,748,"[{'author_name': 'Petter Llanos', 'author_url'..."
